# Hidden Markov Models - Forward Backward

---

## Before Class
In class today we will be implementing the Forward, Backward, and Forward-Backward algorithms. Forward and Backward are very related to viterbi with minor differences in the calculations. 

Prior to class, please do the following:
1. Review these three algorithms in detail

---
## Learning Objectives

1. Forward algorithm
* Backward algorithm
* Forward-Backward algorithm

---
## Background

In the last class we described Markov chains. Here we expand this idea to the concept of a hidden state variable along with observed emissions from the model. We will be using the example of CpG islands from the lecture slides. I have provided the class structure of a simple HMM below. All parameters to this model must be provided as inputs, so essentially this is a class containing the parameters described below:

We define a categorical Hidden Markov Model as $M = (\Sigma, Q, \Theta)$ with the following parameters:

$\Sigma$ : Finite alphabet of symbols (eg. A, C, G, T)

$Q$ : Finite discrete hidden states

$\Theta$: set of probabilities containing: $A$ as transition probabilites $a_{kl}$ for all $k,l \in Q$ and $E$ as emission probabilities $e_k(\sigma)$ for all $k \in Q$ and $\sigma \in \Sigma$ and $B$ as starting probabilities $b_k$ for all $k \in Q$.

We also define a number of $T$ emissions as $y_t = 1 \dots T$ that are drawn from $\Sigma$ and hidden states as $\pi_t = 1 \dots T$ that are drawn from $Q$.

---
## Imports

In [ ]:
import numpy as np
import json

---
## Forward algorithm

The Forward algorithm can be used to estimate the probability of the sequence given our HMM. In general, this is the same as Viterbi except that we sum probablilities instead of taking the max, and we do not need traceback. This is described as bellow:

To estimate $P(x)$, the probability of sequence $x$ given our HMM, calculate forward algorithm values as $f()$ using:
> Initialization ($i = 0$): $f_{k}(0) = e_{k}(\sigma_{0})b_{k}$.<br>
> Recursion ($i = 1 \dots T$): $f_{l}(i) = e_{l}(\sigma_{i})\sum_{k}(f_{k}(i-1)a_{kl})$<br>
> Termination: $P(x) = \sum_{k}f_{k}(T)$

## Backward algorithm
The backward algorithm is essentially the reverse of the forward algorithm. To estimate $P(x)$, the probability of sequence $x$ given our HMM, calculate the backward algorithm values as $r()$ using:
>Initialization ($i = T$): $r_{k}(T) = 1$.<br>
>Recursion ($i = T-1 \dots 1$): $r_{k}(i) = \sum_{l}r_{l}(i+1)a_{kl}e_{l}(\sigma_{i+1})$<br>
>Termination: $P(x) = \sum_{l}r_{k}(1)e_l(\sigma_{1})b_{l}$

## Forward-Backward algorithm
The Forward-Backward algorithm is an extension of both the Forward and Backward algorithms and can be used to estimate the marginal posterior probability of our sequence $x$ being in a state at a specific time. We can calculate this value at every position $i$ and state $k$ as:
>$P(\pi_{i} = k | x) = f_{k}(i)r_{k}(i) / P(x)$


In [ ]:
class HMM(object):
    """Main class for HMM objects
    
    Class for holding HMM parameters and to allow for implementation of
    functions associated with HMMs
    
    Private Attributes:
        _alphabet (set): The alphabet of emissions
        _hidden_states (set): Hidden states in the model
        _transitions (dict(dict)): A dictionary of transition probabilities
        _emissions (dict(dict)): A dictionary of emission probabilities
        _initial (dict): A dictionary of initial state probabilities

    """
    
    __all__ = ['viterbi', 'forward', 'backward', 'forward_backward']

    def __init__(self, alphabet, hidden_states, A=None, E=None, B=None):
        self._alphabet = set(alphabet)
        self._hidden_states = set(hidden_states)
        self._transitions = A
        self._emissions = E
        self._initial = B
        if(self._transitions == None):
            self._initialize_random(self._alphabet, self._hidden_states)
            
    def __str__(self):
        out_text = [f'Alphabet: {self._alphabet}',
                    f'Hidden States: {self._hidden_states}',
                    f'Initial Probabilities: {json.dumps(self._initial, sort_keys = True, indent=4)}',
                    f'Transition Probabilities: {json.dumps(self._transitions, sort_keys = True, indent=4)}',
                    f'Emission Probabilities: {json.dumps(self._emissions, sort_keys = True, indent=4)}']
        return '\n'.join(out_text)
    
    @classmethod
    def __dir__(cls):
        return cls.__all__
        
    def _emit(self, cur_state, symbol):
        return self._emissions[cur_state][symbol]
    
    def _transition(self, cur_state, next_state):
        return self._transitions[cur_state][next_state]
    
    def _init(self, cur_state):
        return self._initial[cur_state]

    def _states(self):
        for k in self._hidden_states:
            yield k
    
    def _get_alphabet(self):
        for sigma in self._alphabet:
            yield sigma
            
    def _initialize_random(self, alphabet, states):
        self._alphabet = set(alphabet)
        self._hidden_states = set(hidden_states)

        #Initialize empty matrices A and E with pseudocounts
        A = {}
        E = {}
        I = {}
        I_rand = np.random.dirichlet(np.ones(len(self._hidden_states)))
        for i, state in enumerate(self._states()):
            E[state] = {}
            A[state] = {}
            I[state] = I_rand[i]
            E_rand = np.random.dirichlet(np.ones(len(self._alphabet)))
            A_rand = np.random.dirichlet(np.ones(len(self._hidden_states)))
            for j, sigma in enumerate(self._get_alphabet()):
                E[state][sigma] = E_rand[j]
            for j, next_state in enumerate(self._states()):
                A[state][next_state] = A_rand[j]
                
        self._transitions = A
        self._emissions = E
        self._initial = I
        return
        
    def viterbi(self, sequence):
        """ The viterbi algorithm for decoding a string using a HMM

        Args:
            sequence (list): a list of valid emissions from the HMM

        Returns:
            result (list): optimal path through HMM given the model parameters
                           using the Viterbi algorithm
        
        Pseudocode for Viterbi:
            Initialization (𝑖=0): 𝑣𝑘(𝑖)=𝑒𝑘(𝜎)𝑏𝑘.
            Recursion (𝑖=1…𝑇): 𝑣𝑙(𝑖)=𝑒𝑙(𝑥𝑖) max𝑘(𝑣𝑘(𝑖−1)𝑎𝑘𝑙); 
                                ptr𝑖(𝑙)= argmax𝑘(𝑣𝑘(𝑖−1)𝑎𝑘𝑙).
            Termination: 𝑃(𝑥,𝜋∗)= max𝑘(𝑣𝑘(𝑙)𝑎𝑘0); 
                             𝜋∗𝑙= argmax𝑘(𝑣𝑘(𝑙)𝑎𝑘0).
            Traceback: (𝑖=𝑇…1): 𝜋∗𝑖−1= ptr𝑖(𝜋∗𝑖).
        """
        pass

    def forward(self, sequence):
        """ The forward algorithm for calculating probability of sequence given HMM

        Args:
            sequence (list): a list of valid emissions from the HMM

        Returns:
            result (float, list of dicts): P(x) and the f matrix as a list
        
        Pseudocode for Forward:
            Initialization (𝑖=0): 𝑓𝑘(0)=𝑒𝑘(𝜎0)𝑏𝑘.
            Recursion (𝑖=1…𝑇): 𝑓𝑙(𝑖)=𝑒𝑙(𝜎𝑖)∑𝑘(𝑓𝑘(𝑖−1)𝑎𝑘𝑙)
            Termination: 𝑃(𝑥)=∑𝑘𝑓𝑘(𝑇)
        """
        pass


    def backward(self, sequence):
        """ The backward algorithm for calculating probability of sequence given HMM

        Args:
            sequence (list): a list of valid emissions from the HMM

        Returns:
            result (float, list of dicts): P(x) and the r matrix as a list
        
        Pseudocode for Backward:
            Initialization (𝑖=T): 𝑟𝑘(𝑇)=1.
            Recursion (𝑖=𝑇−1…1): 𝑟𝑘(𝑖)=∑𝑙𝑟𝑙(𝑖+1)𝑎𝑘𝑙𝑒𝑙(𝜎𝑖+1)
            Termination: 𝑃(𝑥)=∑𝑙𝑟𝑘(1)𝑒𝑙(𝜎1)𝑏𝑙
        """
        pass

    
    def forward_backward(self, sequence):
        """ The forward-backward algorithm for calculating marginal posteriors given HMM

        Args:
            sequence (list): a list of valid emissions from the HMM

        Returns:
            posterior (list of dicts): all posteriors as a list
        
        Pseudocode for Forward-Backward:
            Calculate f[] as forward algorithm
            Calculate r[] as backward algorithm
            for all i in sequence
                for all states
                    posterior[i][state] = f[i][state] * r[i][state] / Px
        """        
        pass


In [ ]:
# This section of code will initialize your HMM with parameters as defined in the lecture slides
# for the identification of CpG Islands.
# All of this should be able to run whether or not you implement the functions!

hidden_states = ('I', 'G') # CpG Island or Genome
alphabet = ('A', 'C', 'G', 'T') # DNA Alphabet

# These are the initial probabilities as defined in the lecture slides
initial_probabilities = {
    'I' : 0.1,
    'G' : 0.9
}


# These are the probabilities of transitioning from outer state to inner state
#  as defined in the lecture slides
transition_probabilities = {
    'I': { 'I' : 0.6, 'G' : 0.4 },
    'G': { 'I' : 0.1, 'G' : 0.9 }
}

# These are the probabilites of each state emmitting each alphabet character
emission_probabilities = {
    'I': { 'A' : 0.1, 'C' : 0.4, 'G' : 0.4, 'T' : 0.1 },
    'G': { 'A' : 0.4, 'C' : 0.1, 'G' : 0.1, 'T' : 0.4 }
}

# Build the model
model = HMM(alphabet, hidden_states, transition_probabilities, emission_probabilities, initial_probabilities)
print(model)

NameError: name 'HMM' is not defined

In [ ]:
# Exact example from slides
sequence = "ACGCGATC"

print ("Forward:")
f_Px, f_matrix = model.forward(list(sequence))
print (f_Px, f_matrix)

print ("\nBackward:")
r_Px, r_matrix = model.backward(list(sequence))
print (r_Px, r_matrix)

print ("\nPosterior:")
posterior = model.forward_backward(list(sequence))
print (posterior)